In [1]:
import os
import pandas as pd
import re

In [2]:
filepath = "ttd_drug_ids.txt"
with open(filepath, encoding="utf-8") as f:
    drug_str = f.read()

In [5]:
pattern = r"(\S*)\tDRUGSMIL\t(\S*)" #r"DRUGSMIL\t(/w+)"
text_to_search = 'D00AAN\tDRUGSMIL\tC1CCN2CCC3C(=CC(CCC=CC1)(C4C3(C2)CC5N4CCCC(=O)CC5)O)C6=NC=CC7=C6NC8=C7C=CC=C8OS(=O)(=O)C9=CC=C(C=C9)Cl'
re.findall(pattern, text_to_search)

[('D00AAN',
  'C1CCN2CCC3C(=CC(CCC=CC1)(C4C3(C2)CC5N4CCCC(=O)CC5)O)C6=NC=CC7=C6NC8=C7C=CC=C8OS(=O)(=O)C9=CC=C(C=C9)Cl')]

In [6]:
drug_list = drug_str.split("_______________________________________________________________________")
len(drug_list)

3

In [7]:
actual_data = drug_list[2]

In [8]:
drug_dict = {
    "DRUG__ID" :[],
    "DRUGSMIL":[]
}

In [9]:
data2 = actual_data.split('\n')
data3 = list(filter(lambda a: a != '\t\t', data2))
data4 = list(filter(lambda a: a != '_\t\t', data3))

In [10]:
for i in data4:
    pattern = r"(\S*)\tDRUGSMIL\t(\S*)"
    drugid = re.findall(pattern, i)
    if drugid !=[]:
        drug_id = drugid[0][0]
        drug_dict["DRUG__ID"].append(drug_id)
        drug_mil = drugid[0][1]
        drug_dict["DRUGSMIL"].append(drug_mil)

In [11]:
len(drug_dict["DRUG__ID"])

21083

In [12]:
len(drug_dict["DRUGSMIL"])

21083

In [13]:
drug_dict

{'DRUG__ID': ['D00AAN',
  'D00AAU',
  'D00ABO',
  'D00ACC',
  'D00ACL',
  'D00ACX',
  'D00AEQ',
  'D00AEX',
  'D00AHD',
  'D00AII',
  'D00AIO',
  'D00AJB',
  'D00AKV',
  'D00AKY',
  'D00AMQ',
  'D00ANJ',
  'D00AOJ',
  'D00AON',
  'D00APD',
  'D00AQR',
  'D00ARM',
  'D00ARN',
  'D00ASA',
  'D00ASH',
  'D00ASL',
  'D00ASZ',
  'D00ATA',
  'D00AUC',
  'D00AUX',
  'D00AVR',
  'D00AVT',
  'D00AWA',
  'D00AWM',
  'D00AWT',
  'D00AXJ',
  'D00AYA',
  'D00AYW',
  'D00BAJ',
  'D00BCG',
  'D00BCO',
  'D00BCP',
  'D00BDC',
  'D00BEA',
  'D00BEM',
  'D00BFF',
  'D00BFT',
  'D00BGN',
  'D00BHL',
  'D00BIE',
  'D00BIY',
  'D00BLD',
  'D00BLL',
  'D00BLY',
  'D00BMF',
  'D00BMI',
  'D00BND',
  'D00BNE',
  'D00BOY',
  'D00BPA',
  'D00BQR',
  'D00BRD',
  'D00BSX',
  'D00BTW',
  'D00BUR',
  'D00BVP',
  'D00BWF',
  'D00BWZ',
  'D00BXG',
  'D00BXK',
  'D00BYM',
  'D00BZS',
  'D00CAF',
  'D00CAO',
  'D00CAU',
  'D00CCI',
  'D00CCV',
  'D00CDF',
  'D00CDM',
  'D00CEG',
  'D00CEN',
  'D00CEO',
  'D00CFB',
  'D

In [14]:
id_df = pd.DataFrame(drug_dict)

In [15]:
id_df = id_df.set_index('DRUG__ID')

In [16]:
id_df.head()

,DRUGSMIL
DRUG__ID,
D00AAN,C1CCN2CCC3C(=CC(CCC=CC1)(C4C3(C2)CC5N4CCCC(=O)...
D00AAU,CCC(C1=CC(=CC=C1)O)C(CC)C2=CC(=CC=C2)O
D00ABO,C1CN(CCN1)C(=O)C2=CC=C(C=C2)C=CC3=NNC4=CC=CC=C43
D00ACC,C1CC2CCC(C3=CC=CC(=C23)C1)N4CCC5(CC4)C(=O)NCN5...
D00ACL,C1=CC(=C(C=C1C(=O)NO)F)CN(C2=NC=CN=C2)C3=NC=CN=C3
